In [2]:
import os
import sys
import pygccxml

from pygccxml import utils
from pygccxml import declarations
from pygccxml import parser
import ctypes
import os


file = open('dllModel.py', 'w')

In [4]:
build_root = os.path.abspath(os.path.join(os.curdir, "Example"))
model = "dllModel"

dll = os.path.join(build_root, model+"_win64.dll")
header = os.path.join(build_root, model+"_ert_shrlib_rtw", model+".h")

assert os.path.exists(dll)
assert os.path.exists(header)

In [5]:
# Setup castxml.
cast_xml_root = os.path.abspath(os.path.join(os.path.curdir, 'castxml'))

In [6]:
# Configure the xml generator
xml_generator_config = parser.xml_generator_configuration_t(
    xml_generator_path=r"castxml\bin\castxml.exe",
    xml_generator="castxml",
    compiler_path=r"C:\MATLAB\SupportPackages\R2015b\MW_MinGW_4_9\bin\gcc.exe")

In [7]:
decls = parser.parse([header], xml_generator_config)
ns = decls[0]

INFO Parsing source file "C:\Projects\python_SimulinkDLL\Example\dllModel_ert_shrlib_rtw\dllModel.h" ... 


In [8]:
def ctypes_cppname(CPPNAME):
    """convert a CPPNAME field to a ctypes type."""
    
    # If the CPPNAME ends with a capital _T then it is a Simulink type (but still defined in the field CPPNAME)
    # Return this as is for use when called in other functions.
    if CPPNAME.endswith('_T'):
        return CPPNAME
    # Current specified Simulink types and their corresponding ctype type.
    type_map = {'long long int': 'c_longlong',
                'long long unsigned int': 'c_ulonglong',
                'short unsigned int': 'c_ushort',
                'int': 'c_int',
                'unsigned int': 'c_uint',
                'long int': 'c_long',
                'long unsigned int': 'c_ulong',
                'char': 'c_char',
                'char': 'c_char',
                'signed char': 'c_byte',
                'unsigned char': 'c_ubyte',
                'float': 'c_float',
                'double': 'c_double',
                'short int': 'c_short'}
    # Loop through the simulink definitions and return if found.
    for cppname, ctype in type_map.items():
        if cppname == CPPNAME:
            # Return ctypes.+ for explicitness and to maintain the namespaces.
            return "ctypes."+ctype
    # If we make it this far raise an exception and investigate the CPPNAME.
    # Fix the function or add another type to the type_map.
    raise(Exception("Unhandled data type: "+CPPNAME))

def var_type(decl_type):
    # If the type has a CPPNAME, just return that type.
    if hasattr(decl_type, 'CPPNAME'):
        return(ctypes_cppname(decl_type.CPPNAME))
    # If the type is an array
    if pygccxml.declarations.is_array(decl_type):
        # And has a CPP Type
        if hasattr(decl_type.base, "CPPNAME"):
            # Get the base type.
            ctype = ctypes_cppname(decl_type.base.CPPNAME)
            # And return that type by the size:
            return "{}*{}".format(ctype.base, decl_type.size)
        elif isinstance(decl_type.base, pygccxml.declarations.cpptypes.declarated_t):
            # If it's a declared type, split the type string off.
            ctype = decl_type.base.decl_string.split(":")[-1]
            return "{}*{}".format(ctype, decl_type.size)
    # If it's a pointer:
    if pygccxml.declarations.is_pointer(decl_type):
        # Remove the pointer reference and get the variable type recursively.
        return "ctypes.POINTER({})".format(var_type(declarations.remove_pointer(decl_type)))
    # IF it's a volatile type:
    if pygccxml.declarations.is_volatile(decl_type):
        # Remove the volatile and get the variable type recursively.
        return "{}".format(var_type(declarations.remove_volatile(decl_type)))
    # If it's a const....
    if pygccxml.declarations.is_const(decl_type):
        return "{}".format(var_type(declarations.remove_const(decl_type)))
    # If it's just a declared type.
    if isinstance(decl_type, pygccxml.declarations.cpptypes.declarated_t):
        # Get the type directly.
        return decl_type.decl_string.split(":")[-1]
    
    # Throw an error if we see something we haven't seen before.
    raise Exception(decl_type)
    
# Get the typedef of a structure.
def struct_tdef(struct, ns):
    for tdef in ns.typedefs():
        try:
            if tdef.decl_type.declaration == struct:
                return tdef.name
        except:
            continue
    return None   

# Get the indentation.
def istr(indent=0,offset=0):
    return " "*(4*indent+offset)

In [9]:
# Print all of the standard type definitions.
print("# Standard Types", file=file)
for typedef in ns.typedefs():
    if hasattr(typedef.decl_type, 'CPPNAME'):
        print("{} = {}".format(typedef.name, ctypes_cppname(typedef.decl_type.CPPNAME)), file=file)

In [10]:
def print_struct(struct, file=sys.stdout, indent=0):
    """ Print a ctypes structure definition.
    """
    # Start defining the class.
    print("{}class {}(ctypes.Structure):".format(istr(indent), struct.name), file=file)
    # Define the fields.
    print("{}_fields_ = [".format(istr(indent+1)), file=file)
    # For each of the variables inside the struct.
    for var in struct.variables():
        # Loop through and define the field name and type.
        print("{}(\"{}\", {}),".format(istr(indent+2),var.name,var_type(var.decl_type)), file=file)
    # Close our brackets.s
    print("{}]".format(istr(indent+1)), file=file)

    # Get the structure typedef
    tdef = struct_tdef(struct, ns)
    if tdef is not None:
        # If it's none and doesn't match the structure name.
        # Because of how Simulink generates the code and how CastXML reads it structures have different 
        # names than their typedef.
        # 
        # Example: 
        #
        # typedef struct Var_tag {
        #     ...
        # } Var_type;
        # 
        # Would have a structure named Var_tag but a typedef named Var_type.
        
        # Create the typedef.
        if tdef != struct.name:
            print("{}{} = {}".format(istr(indent), tdef, struct.name), file=file)

In [12]:
# Get all of the defined structures.
structs = ns.classes(lambda x: declarations.is_struct(x))
# For each of the structures.
for struct in structs:
    # If the structure has no name, pass.
    if struct.name == "":
        continue
    # If locale is in the structure name, pass (They're not any structures we can access in the DLL)
    if 'locale' in struct.name:
        continue

    print_struct(struct, indent=0, file=file)
    print()

externs

Only the variables defined as externs are accessible through the dll. Everything else is internal (and not accessible.)

To grab a reference to the variable we access it with "TYPE.in_dll(dll, VARIABLE_NAME)"

In [13]:
externs = ns.variables(lambda v: v.type_qualifiers.has_extern)
for extern in externs:
#    extern_type = extern.decl_type.declaration.name
    print("{} = {}.in_dll(dll, '{}')".format(extern.name, var_type(extern.decl_type), extern.name), file=file)

In [14]:
file.close()